<a href="https://colab.research.google.com/github/plthiyagu/AI-Engineering/blob/master/09-NLP/Twitter_based_Gender_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.analyticsvidhya.com/blog/2021/08/twitter-based-gender-classification-a-machine-learning-project/?utm_source=feedburner&utm_medium=email&utm_campaign=Feed%3A+AnalyticsVidhya+%28Analytics+Vidhya%29

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')

In [4]:
import nltk

import re 

from nltk.stem import PorterStemmer # for stemming

from nltk.stem import WordNetLemmatizer # for lemmatization

from nltk.corpus import stopwords
nltk.download('punkt')

nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [8]:
df=pd.read_csv('https://raw.githubusercontent.com/plthiyagu/Personnel/master/Dataset/gender_classifier.csv',encoding='latin1')

In [9]:
df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,description,fav_number,gender_gold,link_color,name,profile_yn_gold,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,i sing my own rhythm.,0,NaN,08C2C2,sheezy0,NaN,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,I'm the author of novels filled with family dr...,68,NaN,0084B4,DavdBurnett,NaN,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,louis whining and squealing and all,7696,NaN,ABB8C2,lwtprettylaugh,NaN,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",202,NaN,0084B4,douggarland,NaN,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,37318,NaN,3B94D9,WilfordGemma,NaN,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


In [10]:
df.shape

(20050, 26)

In [11]:
df.drop(['_unit_id','_last_judgment_at','created','fav_number','profileimage','retweet_count','tweet_coord',
         '_trusted_judgments', 'tweet_count', 'tweet_created', 'tweet_id', 'tweet_location', 'user_timezone', 
         '_golden','_unit_state', 'gender_gold', 'link_color', 'name', 'profile_yn_gold', 'sidebar_color', 
         'profile_yn', 'profile_yn:confidence','gender:confidence'], axis=1, inplace=True)

In [12]:
df.head()

,gender,description,text
0,male,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...
1,male,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...
2,male,louis whining and squealing and all,i absolutely adore when louis starts the songs...
3,male,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...
4,female,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...


In [13]:
df.isna().sum()

gender           97
description    3744
text              0
dtype: int64

In [14]:
df.dropna(axis=0,inplace=True)

In [15]:
df['gender'].value_counts()

female     5725
male       5469
brand      4328
unknown     702
Name: gender, dtype: int64

In [16]:
df['gender'] = df[(df['gender'] == 'female') | (df['gender'] == 'male')]

In [17]:
df['gender'].value_counts()

female    5725
male      5469
Name: gender, dtype: int64

In [20]:
for gen in df['gender']:
  if gen=='male':
     df['gender'].replace({'male':'1'},inplace=True)
  elif gen=='female':
     df['gender'].replace({'female':'0'},inplace=True)

df['gender'].value_counts()

0    5725
1    5469
Name: gender, dtype: int64

In [21]:
def clean(review):
  descrip = re.sub('[^a-zA-Z]', ' ', review)
  review = review.lower()
  return review

df['descrip_Cleaned'] = pd.DataFrame(df['description'].apply(lambda x: clean(x)))
df.head()

,gender,description,text,descrip_Cleaned
0,1,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,i sing my own rhythm.
1,1,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...,i'm the author of novels filled with family dr...
2,1,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining and squealing and all
3,1,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,"mobile guy. 49ers, shazam, google, kleiner pe..."
4,0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson the best frontman/kaiser chiefs t...


In [36]:
df['descrip_Cleaned'].replace('[@+]', "", regex=True,inplace=True)
df['descrip_Cleaned'].replace('[()]', "", regex=True,inplace=True)
df['descrip_Cleaned']= df['descrip_Cleaned'].replace('[#+]', "", regex=True)

In [ ]:
url_regex = "(https?://)(s)*(www.)?(s)*((w|s)+.)*([w-s]+/)*([w-]+)((?)?[ws]*=s*[w%&]*)*"
df['descrip_Cleaned'] = df['descrip_Cleaned'].replace(url_regex, "", regex=True)

In [41]:
from nltk.tokenize import word_tokenize
df['descrip_Cleaned'] = [nltk.word_tokenize(tweet) for tweet in df['descrip_Cleaned']]
descrip_new=[]
for each_row in df['descrip_Cleaned']:
  descrip_new.append([i for i in each_row if i.isalpha()])
descrip_new_alpha=[]

stop_words = set(stopwords.words('english'))
for each_row in descrip_new:
  descrip_new_alpha.append([i for i in each_row if i not in stop_words])


In [42]:
description_new_lemma = []
lemma = nltk.WordNetLemmatizer()

for each_row in descrip_new_alpha:
  description_new_lemma.append([lemma.lemmatize(word) for word in each_row])

df['descrip_Cleaned'] = description_new_lemma
df['descrip_Cleaned'] = [" ".join(desc) for desc in df['descrip_Cleaned'].values]


In [43]:
def clean1(review1):
  descrip = re.sub('[^a-zA-Z]', ' ', review1)
  review1 = review1.lower()
  return review1

df['text_Cleaned'] = pd.DataFrame(df['text'].apply(lambda y: clean1(y)))
df.head()

,gender,description,text,descrip_Cleaned,text_Cleaned
0,1,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,sing rhythm,robbie e responds to critics after win against...
1,1,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...,author novel filled family drama romance,ûïit felt like they were my friends and i was...
2,1,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining squealing,i absolutely adore when louis starts the songs...
3,1,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,mobile guy shazam google kleiner perkins yahoo...,hi @jordanspieth - looking at the url - do you...
4,0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson best chief best band xxxx thank k...,watching neighbours on sky+ catching up with t...


In [ ]:
df['text_Cleaned'].replace('[@+]', "", regex=True,inplace=True)
df['text_Cleaned'].replace('[()]', "", regex=True,inplace=True)
df['text_Cleaned']= ['descrip_Cleaned'].replace('[#+]', "", regex=True)
url_regex = "(https?://)(s)*(www.)?(s)*((w|s)+.)*([w-s]+/)*([w-]+)((?)?[ws]*=s*[w%&]*)*" 
df['text_Cleaned'] = df['descrip_Cleaned'].replace(url_regex, "", regex=True)

In [46]:
df['text_Cleaned'] = [nltk.word_tokenize(tweet) for tweet in df['text_Cleaned']]
text_new=[]
for each_row in df['text_Cleaned']:
    text_new.append([i for i in each_row if i.isalpha()])

In [47]:
stop_words = set(stopwords.words('english'))

In [48]:
text_new_alpha=[]
stop_words = set(stopwords.words('english'))

for each_row in text_new:
  text_new_alpha.append([i for i in each_row if i not in stop_words])

In [49]:
text_new_lemma = []
lemma = nltk.WordNetLemmatizer()

for each_row in text_new_alpha:
    text_new_lemma.append([lemma.lemmatize(word) for word in each_row])

In [50]:
df.drop(['description','text'],axis=1,inplace=True)
df.head()

,gender,descrip_Cleaned,text_Cleaned
0,1,sing rhythm,"[robbie, e, responds, to, critics, after, win,..."
1,1,author novel filled family drama romance,"[ûïit, felt, like, they, were, my, friends, a..."
2,1,louis whining squealing,"[i, absolutely, adore, when, louis, starts, th..."
3,1,mobile guy shazam google kleiner perkins yahoo...,"[hi, jordanspieth, -, looking, at, the, url, -..."
4,0,ricky wilson best chief best band xxxx thank k...,"[watching, neighbours, on, sky, catching, up, ..."


In [ ]:
cv = CountVectorizer(max_features = 1000)
x = cv.fit_transform(df['descrip_Cleaned']).toarray()
x1=cv.fit_transform(df['text_Cleaned']).toarray()

In [52]:
A=pd.DataFrame(x)
B=pd.DataFrame(x1)

NameError: ignored

In [53]:
X=pd.concat([B,A],join='outer',axis=1)
X.shape

NameError: ignored

In [54]:
df['gender'].shape

(16224,)

In [55]:
x = np.array(X)

NameError: ignored

In [56]:
y = np.array(df['gender'])

In [57]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 0)

In [ ]:
gnbmodel = GaussianNB()
gnbmodel.fit(X_train , y_train)
y_pred = gnbmodel.predict(X_test)

In [59]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

NameError: ignored

In [60]:
confusion_matrix(y_test, y_pred)

NameError: ignored

In [61]:
print(classification_report(y_test, y_pred))

NameError: ignored

In [62]:
param_grid_nb = {'var_smoothing': np.logspace(0,-9, num=100)}

In [63]:
nbModel_grid = GridSearchCV(estimator=gnbmodel, param_grid=param_grid_nb, verbose=1, cv=3, n_jobs=-1)

NameError: ignored

In [64]:
nbModel_grid.fit(X_train, y_train)

NameError: ignored

In [65]:
nbModel_grid.best_params_

NameError: ignored

In [66]:
 y_pred_hyper = nbModel_grid.predict(X_test)

NameError: ignored

In [67]:
print(confusion_matrix(y_test, y_pred_hyper), ": is the confusion matrix")

NameError: ignored

In [68]:
print(classification_report(y_test, y_pred_hyper))

NameError: ignored

In [69]:
lgbmodel = LGBMClassifier(max_depth=3)
lgbmodel.fit(X_train, y_train)

ValueError: ignored

In [70]:
y_pred1= lgbmodel.predict(X_test)

NotFittedError: ignored

In [71]:
accuracy = accuracy_score(y_test, y_pred1)

print("Accuracy: %.2f%%" % (accuracy * 100.0)

SyntaxError: ignored

In [72]:
confusion_matrix(y_test, y_pred1)

NameError: ignored

In [73]:
print(classification_report(y_test, y_pred1))

NameError: ignored

In [74]:
param_grid = {
              "max_depth": [2, 3, 5, 10],
              "min_child_weight": [0.001, 0.002],
              "learning_rate": [0.05, 0.1]
              }

In [75]:
lgbgrid = GridSearchCV(estimator = lgbmodel, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 0)

lgbgrid.fit(X_train, y_train)


NameError: ignored

In [76]:
lgbgrid.best_params_


NameError: ignored

In [77]:
y_pred1_hyper = lgbgrid.predict(X_test)


NameError: ignored

In [78]:
accuracy = accuracy_score(y_test, y_pred1_hyper)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


NameError: ignored

In [79]:
confusion_matrix(y_test, y_pred1_hyper)


NameError: ignored

In [80]:
print(classification_report(y_test, y_pred1_hyper))


NameError: ignored

In [81]:
xgbmodel = XGBClassifier(max_depth=5, min_child_weight=1)
xgbmodel.fit(X_train, y_train)

TypeError: ignored

In [82]:
y_pred2 = xgbmodel.predict(X_test)


XGBoostError: ignored

In [83]:
confusion_matrix(y_test, y_pred2)


NameError: ignored

In [84]:
print(classification_report(y_test, y_pred2))


NameError: ignored

In [85]:
xgb_param_grid = {
              "max_depth": [3, 5],
              "min_child_weight": [1, 2],
              }


In [86]:
xgbgrid = GridSearchCV(estimator = xgbmodel, param_grid = xgb_param_grid, cv = 3, n_jobs = -1, verbose = 0)

xgbgrid.fit(X_train, y_train)


NameError: ignored

In [87]:
xgbgrid.best_params_


NameError: ignored

In [88]:
y_pred2_hyper = xgbgrid.predict(X_test)

NameError: ignored

In [89]:
accuracy = accuracy_score(y_test, y_pred2_hyper)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


NameError: ignored

In [90]:
confusion_matrix(y_test, y_pred2_hyper)


NameError: ignored

In [91]:
print(classification_report(y_test, y_pred2_hyper))


NameError: ignored